# SPR 2026 - LinearSVC v5 (Tratamento + Aumentação + SMOTE)

**Score baseline:** 0.77885 (4º melhor)

## Melhorias v5:
1. **Tratamento de dados:**
   - Normalização de termos médicos
   - Limpeza de caracteres especiais
   - Padronização de formatos

2. **Aumentação de dados:**
   - SMOTE para classes 5 e 6 (minoritárias)

3. **Melhores práticas:**
   - TF-IDF 15k features, ngram 1-2, sublinear_tf
   - class_weight='balanced'
   - CalibratedClassifierCV para probabilidades

**Meta:** Superar 0.78+ F1-Macro

---
## CONFIGURAÇÃO KAGGLE:
1. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
2. **Settings** → Internet → **OFF**
---

In [ ]:
# ===== SPR 2026 - LINEARSVC v5 (TRATAMENTO + SMOTE) =====

import os
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - LinearSVC v5 (Tratamento + SMOTE)")
print("="*60)

SEED = 42
DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'

# ========== VERIFICAR DATASET PRIMEIRO ==========
if not os.path.exists(DATA_DIR):
    print("\n" + "="*60)
    print("ERRO: Dataset não encontrado!")
    print("="*60)
    print("\nAdicione o dataset:")
    print("Add Input -> Competition -> spr-2026-mammography-report-classification")
    raise FileNotFoundError(f"Dataset não encontrado: {DATA_DIR}")
print(f"Dataset: {DATA_DIR}")

np.random.seed(SEED)

# ========== CONFIGURAÇÕES ==========
USE_SMOTE = True
SMOTE_TARGET_5 = 500
SMOTE_TARGET_6 = 500
USE_CALIBRATION = True

In [ ]:
# ========== FUNÇÕES DE TRATAMENTO DE DADOS ==========
print("\n[1/6] Definindo funções de tratamento...")

def clean_text(text):
    """Limpeza básica do texto."""
    if pd.isna(text):
        return ""
    text = str(text)
    # Normalizar espaços
    text = re.sub(r'\s+', ' ', text)
    # Remover caracteres de controle
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    return text.strip()

def normalize_medical_terms(text):
    """Normaliza termos médicos comuns em mamografia."""
    # Normalizar variações de BI-RADS
    text = re.sub(r'bi-?rads?', 'BIRADS', text, flags=re.IGNORECASE)
    text = re.sub(r'birads\s*(\d)', r'BIRADS \1', text, flags=re.IGNORECASE)
    
    # Normalizar termos anatômicos
    text = re.sub(r'qse|quadrante\s*superior\s*externo', 'quadrante_superior_externo', text, flags=re.IGNORECASE)
    text = re.sub(r'qsi|quadrante\s*superior\s*interno', 'quadrante_superior_interno', text, flags=re.IGNORECASE)
    text = re.sub(r'qie|quadrante\s*inferior\s*externo', 'quadrante_inferior_externo', text, flags=re.IGNORECASE)
    text = re.sub(r'qii|quadrante\s*inferior\s*interno', 'quadrante_inferior_interno', text, flags=re.IGNORECASE)
    
    # Normalizar achados
    text = re.sub(r'micro-?calcifica[çc][õo]es', 'microcalcificacoes', text, flags=re.IGNORECASE)
    text = re.sub(r'n[oó]dulo', 'nodulo', text, flags=re.IGNORECASE)
    text = re.sub(r'assimetria\s*focal', 'assimetria_focal', text, flags=re.IGNORECASE)
    text = re.sub(r'distorção\s*arquitetural', 'distorcao_arquitetural', text, flags=re.IGNORECASE)
    
    # Criar features para negações (importante para classificação)
    text = re.sub(r'ausência\s*de|sem\s*evidência\s*de|não\s*se\s*observa', 'NEGACAO', text, flags=re.IGNORECASE)
    
    return text.lower()

def preprocess_text(text):
    """Pipeline completo de pré-processamento."""
    text = clean_text(text)
    text = normalize_medical_terms(text)
    return text

print("Funções de tratamento definidas!")

In [ ]:
# ========== CARREGAR E TRATAR DADOS ==========
print("\n[2/6] Carregando e tratando dados...")

train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {train.shape} | Test: {test.shape}")

# Auto-detectar colunas
def find_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
        for col in df.columns:
            if col.lower() == c.lower():
                return col
    return None

TEXT_COL = find_col(train, ['report', 'text', 'laudo', 'texto', 'content'])
LABEL_COL = find_col(train, ['target', 'label', 'birads', 'classe', 'class'])
ID_COL = find_col(test, ['ID', 'id', 'Id', 'index', 'idx'])
print(f"Colunas: texto={TEXT_COL}, label={LABEL_COL}, id={ID_COL}")

# Aplicar tratamento
print("\nAplicando tratamento de texto...")
train['text_processed'] = train[TEXT_COL].apply(preprocess_text)
test['text_processed'] = test[TEXT_COL].apply(preprocess_text)

# Mostrar exemplo
print("\nExemplo de tratamento:")
print(f"Original: {train[TEXT_COL].iloc[0][:150]}...")
print(f"Tratado:  {train['text_processed'].iloc[0][:150]}...")

# Distribuição de classes
print("\nDistribuição de classes:")
print(train[LABEL_COL].value_counts().sort_index())

In [ ]:
# ========== TF-IDF ==========
print("\n[3/6] TF-IDF...")

tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train = tfidf.fit_transform(train['text_processed'])
X_test = tfidf.transform(test['text_processed'])
y_train = train[LABEL_COL].values

print(f"TF-IDF shape: {X_train.shape}")

In [ ]:
# ========== SMOTE ==========
print("\n[4/6] SMOTE para classes minoritárias...")

if USE_SMOTE:
    try:
        from imblearn.over_sampling import SMOTE
        
        # Verificar contagem atual
        unique, counts = np.unique(y_train, return_counts=True)
        print("Antes do SMOTE:")
        for u, c in zip(unique, counts):
            print(f"  Classe {u}: {c}")
        
        # Aplicar SMOTE
        smote = SMOTE(
            sampling_strategy={5: SMOTE_TARGET_5, 6: SMOTE_TARGET_6},
            random_state=SEED,
            k_neighbors=3
        )
        X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
        
        print(f"\nApós SMOTE: {X_train_smote.shape[0]} amostras")
        unique, counts = np.unique(y_train_smote, return_counts=True)
        for u, c in zip(unique, counts):
            print(f"  Classe {u}: {c}")
            
        X_train_final = X_train_smote
        y_train_final = y_train_smote
        
    except ImportError:
        print("imblearn não disponível, usando dados originais")
        X_train_final = X_train
        y_train_final = y_train
else:
    X_train_final = X_train
    y_train_final = y_train
    print("SMOTE desabilitado")

In [ ]:
# ========== TREINAMENTO ==========
print("\n[5/6] Treinando LinearSVC...")

# Modelo base
base_svc = LinearSVC(
    C=1.0,
    class_weight='balanced',
    random_state=SEED,
    max_iter=5000,
    tol=1e-4,
    dual=True
)

if USE_CALIBRATION:
    print("Usando CalibratedClassifierCV...")
    model = CalibratedClassifierCV(base_svc, cv=5, method='sigmoid')
else:
    model = base_svc

# Cross-validation
print("\nCross-validation (5-fold)...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
cv_scores = cross_val_score(base_svc, X_train_final, y_train_final, cv=cv, scoring='f1_macro')
print(f"CV F1-Macro: {cv_scores.mean():.5f} (+/- {cv_scores.std()*2:.5f})")

# Treinar modelo final
print("\nTreinando modelo final...")
model.fit(X_train_final, y_train_final)
print("Modelo treinado!")

In [ ]:
# ========== PREDIÇÃO E SUBMISSION ==========
print("\n[6/6] Gerando predições...")

predictions = model.predict(X_test)

# Gerar submission
submission = pd.DataFrame({
    ID_COL: test[ID_COL],
    LABEL_COL: predictions
})

submission.to_csv('submission.csv', index=False)
print(f"\nSubmission salva: submission.csv")
print(submission.head())
print(f"\nDistribuição das predições:")
print(pd.Series(predictions).value_counts().sort_index())